In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Aufbau eines Nachfrageprognosemodells mithilfe von BigQuery ML

In diesem Notebook zeigen wir, wie mithilfe von BigQuery ML ein Zeitreihenmodell trainiert, ausgewertet und eingesetzt werden kann. Es bietet eine End-to-End-Lösung für die Vorhersage mehrerer Produkte. Unter Verwendung des öffentlichen Datensatzes [Iowa Liquor Sales](https://console.cloud.google.com/marketplace/details/obfuscated-ga360-data/obfuscated-ga360-data?filter=solution-type:dataset) erstellen wir mit wenigen SQL-Abfragen fünf Zeitreihenmodelle, wobei jedes Modell den Einzelhandelsumsatz eines einzelnen Spirituosenprodukts vorhersagt. 

Am Ende dieses Notebooks haben wir gelernt, wie man:
* Zeitreihendaten in das richtige Format vorbereitet, das für die Erstellung des Modells erforderlich ist.
* Ein Zeitreihenmodell in BigQuery ML trainiert.
* Das Modell auswertet.
* Vorhersagen über die zukünftige Nachfrage mithilfe des Modells trifft.


## Setup

### Notwendige Bibliotheken

In [ ]:
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import pandas as pd
from google.cloud import bigquery

### Plotting-Skript (für später)

In [ ]:
def plot_historical_and_forecast(input_timeseries, 
                                 timestamp_col_name, 
                                 data_col_name, 
                                 forecast_output=None, 
                                 actual=None, 
                                 title=None,
                                 plotstartdate=None):

    if plotstartdate:
        input_timeseries[timestamp_col_name] = pd.to_datetime(input_timeseries[timestamp_col_name])
        input_timeseries = input_timeseries[input_timeseries[timestamp_col_name] >= pd.to_datetime(plotstartdate)]
        
    input_timeseries = input_timeseries.sort_values(timestamp_col_name)    
    
    # Plot the input historical data
    plt.figure(figsize=(20,6))
    plt.plot(input_timeseries[timestamp_col_name], input_timeseries[data_col_name], label = 'Historical')
    plt.xlabel(timestamp_col_name)
    plt.ylabel(data_col_name)

    if forecast_output is not None:
        forecast_output = forecast_output.sort_values('forecast_timestamp')
        forecast_output['forecast_timestamp'] = pd.to_datetime(forecast_output['forecast_timestamp'])
        x_data = forecast_output['forecast_timestamp']
        y_data = forecast_output['forecast_value']
        confidence_level = forecast_output['confidence_level'].iloc[0] * 100
        low_CI = forecast_output['confidence_interval_lower_bound']
        upper_CI = forecast_output['confidence_interval_upper_bound']
        # Plot the forecast data
        plt.plot(x_data, y_data, alpha = 1, label = 'Forecast', linestyle='--')
        # Shade the confidence interval
        plt.fill_between(x_data, low_CI, upper_CI, color = '#539caf', alpha = 0.4, 
                         label = f'{confidence_level} confidence interval')

    # Plot actual data
    if actual is not None:
        actual = actual.sort_values(timestamp_col_name)
        plt.plot(actual[timestamp_col_name], actual[data_col_name], label = 'Actual', linestyle='--')   

    # Display title, legend
    plt.title(f'{title}', fontsize= 16)
    plt.legend(loc = 'upper center', prop={'size': 16})

### Ein BigQuery Dataset erstellen

Wir müssen `US` als location angeben, um Daten aus dem public dataset in unser Dataset kopieren zu können, welches sich ebenfalls in US befindet.

In [ ]:
%%bigquery
CREATE SCHEMA IF NOT EXISTS
bqmlforecast
OPTIONS (
    location="US"
)

## Trainingsdaten vorbereiten

Wir trainieren die Zeitreihenmodelle auf einem Datensatz mit Transaktionsdaten. Jede Zeile stellt eine Transaktion für ein einzelnes Produkt dar, das durch den Wert `item_description` identifiziert wird, und enthält Details wie die Anzahl der verkauften Flaschen und den Verkaufsbetrag in Dollar. In den folgenden Schritten verwenden wir den Wert für die Anzahl der verkauften Flaschen zur Prognose der Produktnachfrage.

In [ ]:
%%bigquery

SELECT 
    invoice_and_item_number,
    date,
    store_number,
    item_description,
    bottles_sold,
    sale_dollars
FROM
  `bigquery-public-data.iowa_liquor_sales.sales` 
LIMIT 
  5

### Start- und Enddatum für die Trainingsdaten festlegen

Wir können die Parameter `TRAININGDATA_STARTDATE` und `TRAININGDATA_ENDDATE` anpassen, um das Start-/Enddatum der Trainingsdaten anzugeben:

In [ ]:
ARIMA_PARAMS = {
    'TRAININGDATA_STARTDATE': '2020-01-01',
    'TRAININGDATA_ENDDATE': '2022-01-01',
}
ARIMA_PARAMS

### Trainingsdaten in einen Tabelle schreiben

Einige Tage in den Daten weisen keine Transaktionen für ein bestimmtes Produkt auf. BigQueryML kann automatisch einige typische Vorverarbeitungen durchführen:

* Fehlende Werte: Die Werte werden mit lokaler linearer Interpolation imputed.
* Duplizierte Zeitstempel: Die Werte werden über duplizierte Zeitstempel gemittelt.
* Spike- und Dip-Anomalien: Die Werte werden anhand lokaler z-Scores standardisiert.

In [ ]:
%%bigquery --params $ARIMA_PARAMS

CREATE OR REPLACE TABLE bqmlforecast.training_data AS (
    WITH topsellingitems AS(
         SELECT 
            item_description,
            count(item_description) cnt_transactions
        FROM
            `bigquery-public-data.iowa_liquor_sales.sales` 
        GROUP BY 
            item_description
        ORDER BY cnt_transactions DESC
        LIMIT 5
    )
    SELECT 
        date,
        item_description AS item_name,
        SUM(bottles_sold) AS total_amount_sold
    FROM
        `bigquery-public-data.iowa_liquor_sales.sales` 
    GROUP BY
        date, item_name
    HAVING 
        date BETWEEN @TRAININGDATA_STARTDATE AND @TRAININGDATA_ENDDATE
        AND item_description IN (SELECT item_description FROM topsellingitems)
    );

SELECT 
    date,
    item_name,
    total_amount_sold
FROM 
    bqmlforecast.training_data 
ORDER BY item_name, date
LIMIT 10

### Verkaufshistorie der Zielspirituosen

Wir speichern die Trainingsdaten im Pandas-Dataframe `pdfhistorical`:

In [ ]:
%%bigquery dfhistorical

SELECT * FROM bqmlforecast.training_data

In [ ]:
itemslist = list(dfhistorical.item_name.unique())

for item in itemslist:
    
    datah = dfhistorical[dfhistorical.item_name==item]
    plot_historical_and_forecast(input_timeseries = datah, 
                                 timestamp_col_name = "date",
                                 data_col_name = "total_amount_sold", 
                                 forecast_output = None, 
                                 actual = None,
                                 title = item)

## Modelltraining

Da das Modell für mehrere Produkte in einem einzigen Modellerstellung-Statement trainiert wird, geben wir die Spalte `item_name` für den Parameter [TIME_SERIES_ID_COL](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create-time-series#time_series_id_col) an. Für einen einzelnen Zielartikel brauchen wir diese Angabe nicht.
Weitere Informationen zum SQL-Statement zur Modellerstellung: [Dokumentation zur Erstellung von BigQuery ML-Zeitreihenmodellen](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create-time-series#create_model_syntax).

Ein häufiges Problem bei Zeitreihendaten sind Feiertagseffekte und Saisonalitäten. BigQueryML kann das durch die Angabe der `HOLIDAY_REGION` berücksichtigen. Standardmäßig ist die Modellierung von Feiertagseffekten deaktiviert. Bei aktivierten Feiertagseffekten werden Ausreißer, die während der Feiertage auftreten, nicht mehr als Anomalien behandelt. Da wir hier Daten aus Iowa analysieren, setzen wir die `HOLIDAY_REGION` auf `US`. Es gibt aber auch Feiertagsdaten für andere Länder.

In [ ]:
%%bigquery

CREATE OR REPLACE MODEL bqmlforecast.arima_model

OPTIONS(
  MODEL_TYPE='ARIMA',
  TIME_SERIES_TIMESTAMP_COL='date', 
  TIME_SERIES_DATA_COL='total_amount_sold',
  TIME_SERIES_ID_COL='item_name',
  HOLIDAY_REGION='US'
) AS

SELECT 
    date,
    item_name,
    total_amount_sold
FROM
  bqmlforecast.training_data

### Modellevaluation

Wir können [ML.EVALUATE](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-evaluate) verwenden, um die Qualitätsmetriken aller erstellten Modelle anzuzeigen. 

Die Spalten `non_seasonal_`{`p`,`d`,`q`} und `has_drift` definieren das Zeitreihenmodell. Die Spalten `log_likelihood`, `AIC` und `variance` sind für das Model Fitting relevant. Beim Model Fitting wird das beste Modell mit Hilfe des [auto.ARIMA-Algorithmus](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create-time-series#auto_arima) ermittelt, und zwar für jede Zeitreihe einzeln.

In [ ]:
%%bigquery
SELECT * FROM ML.EVALUATE(MODEL bqmlforecast.arima_model)

Wir können sehen, dass fünf Modelle trainiert wurden, eines für jedes Produkt. Jedes Modell hat seine eigenen Hyperparameter, und die erkannte Saisonalität für diese fünf Modelle ist `WEEKLY` und teilweise `YEARLY`.

## Modellvorhersagen treffen

Wir können mit [ML.FORECAST](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-forecast) Vorhersagen treffen, die die nächsten _n_ Werte vorhersagen, wie im Parameter `horizon` angegeben. Wir können auch optional das `confidence_level` ändern, um den Prozentsatz der zukünftigen Werte zu ändern, die in das Vorhersageintervall fallen. Die Vorhersagedaten werden im DataFrame `dfforecast` gespeichert, damit sie in einem späteren Schritt dargestellt werden können.


In [ ]:
%%bigquery dfforecast

DECLARE HORIZON STRING DEFAULT "30";
DECLARE CONFIDENCE_LEVEL STRING DEFAULT "0.90";

EXECUTE IMMEDIATE format("""
    SELECT
      *
    FROM 
      ML.FORECAST(MODEL bqmlforecast.arima_model, 
                  STRUCT(%s AS horizon, 
                         %s AS confidence_level)
                 )
    """,HORIZON,CONFIDENCE_LEVEL)

In [ ]:
dfforecast.head()

Da `horizon` auf 30 gesetzt ist, ist das Ergebnis 30 x (Anzahl der Produkte), mit einer Zeile pro vorhergesagtem Wert:

In [ ]:
print(f"Number of rows: {dfforecast.shape[0]}")

#### Vorhersagen visualiseren

Wir können die Vorhersagen an die Zeitreihen anhängen und so einen optischen Eindruck bekommen, ob die Vorhersagen plausibel sind:

In [ ]:
itemslist = list(dfhistorical.item_name.unique())

for item in itemslist:
    datah = dfhistorical[dfhistorical.item_name==item].copy()
    dataf = dfforecast[dfforecast.item_name==item].copy()
    
    plot_historical_and_forecast(input_timeseries = datah, 
                                 timestamp_col_name = "date", 
                                 data_col_name = "total_amount_sold",
                                 forecast_output = dataf, 
                                 actual = None,
                                 title = item,
                                 plotstartdate = "2021-01-01")

#### Mit den tatsächlichen Verkäufen vergleichen

Wir extrahieren zunächst die tatsächlichen Verkäufe aus dem Vergleichszeitraum

In [ ]:
%%bigquery dfactual --params $ARIMA_PARAMS

DECLARE HORIZON STRING DEFAULT "30";

SELECT 
    date,
    item_description AS item_name,
    SUM(bottles_sold) AS total_amount_sold
FROM
    `bigquery-public-data.iowa_liquor_sales.sales` 
GROUP BY
    date, item_name
HAVING 
    date BETWEEN DATE_ADD(@TRAININGDATA_ENDDATE, 
                              INTERVAL 1 DAY) 
            AND DATE_ADD(@TRAININGDATA_ENDDATE, 
                             INTERVAL 1+CAST(HORIZON AS INT64) DAY) 
ORDER BY
    date;

Wir fügen die tatsächlichen Verkäufe in die obige Visualisierung ein.

In [ ]:
itemslist = list(dfhistorical.item_name.unique())

for item in itemslist:
    datah = dfhistorical[dfhistorical.item_name==item].sort_values('date')
    dataf = dfforecast[dfforecast.item_name==item].sort_values(['forecast_timestamp'])
    dataa = dfactual[dfactual.item_name==item].sort_values('date')
    plot_historical_and_forecast(input_timeseries = datah, 
                             timestamp_col_name = "date", 
                             data_col_name = "total_amount_sold", 
                             forecast_output = dataf, 
                             actual = dataa,
                             title = item,
                             plotstartdate = "2021-06-01")